# Init

In [1]:
import configparser, sys

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import kaiser

sys.path.insert(0, "../")
import toolsdiadem.tools as dia
import toolsdiadem.ser as ser
from toolsdiadem.ConfigSer import ConfigSer
from toolsdiadem.Gate import Gate

In [2]:
%matplotlib qt5
%load_ext autoreload
%autoreload 2

In [67]:
cmap='jet'
configFilename = f"C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR10_plaqueSupport.ini"
configRef = f"C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR10_plaqueRef.ini"
conf = ConfigSer(configFilename)
confRef = ConfigSer(configRef)
conf.printConfig()
confRef.printConfig()

centerFreq = 92.5
bandWidth = 17.4
idxElevation = 21 # elevation correspondant au maximum releve avec la plaque support
vmin = -90
vmax = -40

C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR10_plaqueSupport.ini
nb_elev 41
nb_freq 2801
nb_ssb 4
el0 0.65
az0 -0.44
base_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2
out_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/processing/wr10
gateWidth 500.0
gateCenter 6650.0
C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/WR10_plaqueRef.ini
nb_elev 41
nb_freq 2801
nb_ssb 4
el0 0.275
az0 -0.44
base_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2
out_path C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/processing/wr10
gateWidth 0.0
gateCenter 0.0


## Fond de chambre

In [58]:
file_f2c = f"{conf.out_path}/data/S11_fond_de_chambre.txt"
tmp = np.loadtxt(file_f2c, skiprows=1)
print(file_f2c)
S11_f2c = tmp[:,1] + 1j * tmp[:,2]

C:/Users/pleroy/Documents/CAMILL/SER/WR15_v2/processing/wr10/data/S11_fond_de_chambre.txt


In [59]:
fig, ax = plt.subplots(1,1)
plt.plot(ser.dB(np.fft.ifft(S11_f2c)), label="fond de chambre")
plt.grid()
plt.legend()

## Probe

In [60]:
tmp = np.loadtxt(f"{conf.out_path}/data/S11_probe_interpolated.txt")
S11_probe = tmp[:,1] + 1j * tmp[:,2]
freq_probe = tmp[:,0]

## Plaque de référence

In [61]:
# THE DATA STORED AFTER PROCESSING ARE ALERADY CONJUGATED
tmp = np.loadtxt(f"{conf.out_path}/data/S11_plaque_ref_from_scan.txt", skiprows=1)
S11_reference = tmp[:,1] + 1j * tmp[:,2]
freq_all = tmp[:,0]
d = np.linspace(0, 300 / (freq_all[1] - freq_all[0]), conf.nb_freq) * 0.5

In [62]:
fig, ax = plt.subplots(1,1)
plt.plot(ser.dB(np.fft.ifft(S11_reference)), label="reference" )
plt.plot(ser.dB(np.fft.ifft(S11_f2c)), label="fond de chambre" )
plt.grid()
plt.legend()

In [63]:
fig, ax = plt.subplots(1,1)
ax.plot(d, ser.dB(np.fft.ifft(S11_reference)), label="reference")
ax.plot(d, ser.dB(np.fft.ifft(S11_f2c)), label="f2c")
ax.plot(d, ser.dB(np.fft.ifft(S11_probe)), label="probe")
ax.plot(d, ser.dB(np.fft.ifft(S11_reference - S11_probe)), label="reference - probe")
ax.plot(d, ser.dB(np.fft.ifft(S11_reference - S11_f2c)), label="reference - f2c")
ax.grid()
ax.legend()
ax.set_xlabel("[m]")
ax.set_ylabel("[dB]")
ax.set_title( "time domain" )

Text(0.5, 1.0, 'time domain')

# Réflectivité
ref: ser4siepel_multiband_scan_elev_Reflectivite.sce

In [10]:
myGate = Gate(d, width=conf.gateWidth, center=conf.gateCenter)
gate = myGate.gateLLC

gateSize 233


## Lecture des données

In [41]:
date, absorbant = "20200715", "039_W_u"
#date, absorbant = "20200715", "040_W_u_v2"
dir_abs = f"{conf.base_path}/{date}/{absorbant}"
freq_abs, S11_abs = ser.getData_ssb(conf.nb_elev, conf.nb_freq, conf.nb_ssb, dir_abs, tag="WR10_")

## Calcul de la réflectivité

In [42]:
reflectivite = ser.getReflectivite(S11_abs, S11_reference, S11_f2c, gate)

In [43]:
toPlot = np.zeros((conf.nb_elev, conf.nb_freq))
gatebis  = 0.0 * gate
gatebis[np.where(np.abs(freq_abs - centerFreq) < bandWidth)] = 1.0
toPlot = 20 * np.log10(1e-12 + np.abs(reflectivite * gatebis))

valMin = -90
toPlot = 0.5 * (np.abs(toPlot - valMin) + toPlot - valMin) + valMin
valMax = np.amax(toPlot)
valMin = np.amin(toPlot)
print(f"valMax = {valMax:.2f} valMin = {valMin:.2f}")

fig, ax = plt.subplots(1, 1)
im = plt.pcolormesh(conf.elevation, np.flip(freq_abs), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax)
title = f"Réflectivité absorbant {absorbant}, {date}"
ax.set_title(title)
ax.set_xlabel("Elévation [°]")
ax.set_ylabel("Fréquence [GHz]")
dia.addColorBar(im, ax, aspect='auto')

valMax = -54.70 valMin = -90.00


In [25]:
descriptor = f"{absorbant}_{date}"
filename = f"{conf.out_path}/data/reflectivite_{descriptor}"
np.save( filename, reflectivite )

In [30]:
fig, ax = plt.subplots(1,1)
ax.plot(freq_abs, toPlot[idxElevation, :])
ax.grid()
title = f"Réflectivité [{absorbant}]"
ax.set_title(title)
ax.set_xlabel("f [GHz]")
ax.set_ylabel("réflectivité [dB]")

Text(0, 0.5, 'réflectivité [dB]')

In [31]:
fig, ax = plt.subplots(1,1)
ax.plot(gatebis)

In [34]:
np.save( f"{conf.out_path}/data/freq_WR10", freq_abs )
np.save( f"{conf.out_path}/data/elevation_WR10", conf.elevation )

# Tous les absorbants

In [69]:
idx = np.where(np.abs(freq_abs - centerFreq) < bandWidth)
idxAlt = np.where(np.abs(freq_abs - centerFreq) >= bandWidth)

dic_stats = {}
toPlot = np.zeros((conf.nb_elev, conf.nb_freq))

gate = myGate.gateLLC
date = "20200715" # 09, 15, 16

for absorbant in conf.dic_dir[date]:
    
    seq = 0
    flag_gate = 1
    data_dir = f"{conf.base_path}/{date}/{absorbant}"
    descriptor = f"{absorbant}_{date}"
    
    print(f"======= {descriptor} =======")
    
    ############## 16 07 2020 ##############
    ############## 16 07 2020 ##############
    
    freq_abs, S11 = ser.getData_ssb(conf.nb_elev, conf.nb_freq, conf.nb_ssb, data_dir, tag="WR10_")
            
    #=============
    # REFLECTIVITE
    reflectivite = ser.getReflectivite(S11, S11_reference, S11_f2c, gate)
    np.save( 
        f"{conf.out_path}/data/reflectivite/{descriptor}",
        reflectivite 
    )
    
    # drop lower and higher frequencies values
    toPlot[:,idx] = 20 * np.log10(1e-12 + np.abs( reflectivite[:,idx]))
    toPlot[:,idxAlt] = -90

    fig, ax = plt.subplots( 1, 1 )
    im = ax.pcolormesh(conf.elevation, np.flip( freq_abs ), np.rot90(toPlot), cmap=cmap, vmin=vmin, vmax=vmax)
    title = f"Réflectivité [{descriptor}]"
    ax.set_title(title)
    ax.set_xlabel( "Elévation [°]" )
    ax.set_ylabel( "Fréquence [GHz]" )
    dia.addColorBar(  im, ax, aspect='auto' )
    fig.savefig(
        f"{conf.out_path}/plot/reflectivite/reflectivite_{descriptor}.png"
    )

    #======
    # STATS
    valMax = np.amax( toPlot )
    valMin = np.amin( toPlot )

    absReflectivite   = np.abs( reflectivite )
    reflectiviteMoy   = np.average( absReflectivite[:,idx] )
    reflectiviteSigma = np.std( absReflectivite[:,idx] )

    reflectiviteMoy_dB   = 20 * np.log10( np.abs( reflectiviteMoy ) )
    reflectiviteSigma_dB = 20 * np.log10( np.abs( reflectiviteSigma ) )
        
    #==========
    # HISTOGRAM
    fig, ax = plt.subplots(1,1)
    ax.hist((absReflectivite[:,idx]).flatten(), 50, label="histogramme (valeurs linéaires)")
    ax.grid()
    title = f"Réflectivité, histogramme [{descriptor}]"
    ax.set_title(title)
    ax.legend()
    fig.savefig(
        f"{conf.out_path}/plot/hist/hist_{descriptor}.png"
    )
        
    #===========
    # PLOT WRT F
    fig, ax = plt.subplots(1,1)
    ax.plot( freq_abs, toPlot[ idxElevation, : ] )
    ax.grid()
    title = f"Réflectivité [{descriptor}]"
    ax.set_title(title)
    ax.set_xlabel("f [GHz]")
    ax.set_ylabel("réflectivité [dB]")
    fig.savefig(
        f"{conf.out_path}/plot/reflectivite_vs_f/reflectivite_vs_f_{descriptor}.png"
    )
    
    plt.close("all")
    
    dic_stats[f"{descriptor}"] = (valMax, valMin, reflectiviteMoy, reflectiviteSigma, reflectiviteMoy_dB)

======= 038_E_20200715 =======
======= 040_W_u_v2_20200715 =======
======= 041_V_u_20200715 =======
======= 042_G_20200715 =======
======= 043_L_20200715 =======
======= 044_M_20200715 =======
======= 045_X_20200715 =======
======= 046_O_20200715 =======
======= 047_N_20200715 =======
======= 048_K_20200715 =======


In [ ]:
for absorbant in dic_stats.keys():
    stats = dic_stats[absorbant]
    print(f"{absorbant} {stats[0]:.2f} {stats[1]:.2f} {stats[2]:.6f} {stats[3]:.6f} {stats[4]:.2f}")

# Remove all

In [ ]:
for root, dirs, files in os.walk(f"{base_path}/processing"):
    for f in files:
        if f.endswith(".png") or f.endswith(".npy"):
            os.remove(os.path.join(root, f)) # remove or unlink does the same

In [ ]:
os.walk(f"{base_path}/processing")

# getAzSlRoElPolFFPolCATR

In [ ]:
el = ser.getAzSlRoElPolFFPolCATR( "/home/pleroy/DATA/DIADEM/SIEPEL/SER/WR10/190910/Plaque" )[:,3]
print(f"elevation goes from {np.amin(el)} to {np.amax(el)}")